Importação dos dados que estão presente em um repositório no Github no formato de arquivos .csv

In [1]:
import pandas as pd 
import numpy as np

In [2]:
x1 = pd.read_csv('https://github.com/ascdavia/DeepLearning_SignalProcessing-sEMG/blob/main/Database/sEMG_Basic_Hand_movements_upatras_csv_files/Database_1/df1_mov_all.csv?raw=true', compression = None)
x = x1.drop(x1.columns[0], axis=1)

In [ ]:
x = x.values.reshape(x.shape[0], x.shape[1], 1)
print(x.shape)

(900, 3000, 1)


Definição da matriz de saída *y*

In [ ]:
base = np.ones((150,1), dtype=np.int64)
mov_cyl = base*0
mov_hook = base*1
mov_lat = base*2
mov_palm = base*3
mov_spher = base*4
mov_tip = base*5

In [ ]:
y = np.vstack([mov_cyl,mov_hook,mov_lat,mov_palm,mov_spher,mov_tip])

Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
treinoX, testeX, treinoY, testeY = train_test_split(x, y, test_size=0.15, random_state=42)

Leave One Out - CNN 1D

In [ ]:
import tensorflow as tf
import keras
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import GlobalAveragePooling1D
from keras.layers import Flatten

from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import accuracy_score

In [ ]:
num_classes = 6

model = keras.models.Sequential()
model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=(treinoX.shape[1], treinoX.shape[2])))
model.add(Conv1D(64, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
#model.summary()

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
loo = LeaveOneOut()

In [ ]:
x2 = treinoX
y2 = treinoY
loo.get_n_splits(x2)

765

In [ ]:
y_true, y_pred = list(), list()

In [ ]:
i = 0

for train_index, test_index in loo.split(x2): 
  X_train, X_test = x2[train_index], x2[test_index]
  y_train, y_test = y2[train_index], y2[test_index]
  
  # fit model
  model.fit(X_train, y_train, batch_size=16, epochs=10)

  print('Interação:')
  print(i)
  i = i+1
  #evaluete model
  #yhat = model.predict(X_test)
  #y_true.append(y_test[0])
  #y_pred.append(yhat[0])

Streaming output truncated to the last 5000 lines.
48/48 [==============================] - 8s 162ms/step - loss: 7.3922e-06 - accuracy: 1.0000
Epoch 2/5
48/48 [==============================] - 8s 162ms/step - loss: 6.0992e-06 - accuracy: 1.0000
Epoch 3/5
48/48 [==============================] - 8s 163ms/step - loss: 6.0727e-06 - accuracy: 1.0000
Epoch 4/5
48/48 [==============================] - 8s 164ms/step - loss: 5.5079e-06 - accuracy: 1.0000
Epoch 5/5
48/48 [==============================] - 8s 162ms/step - loss: 6.9728e-06 - accuracy: 1.0000
Interação:
21
Epoch 1/5
48/48 [==============================] - 8s 163ms/step - loss: 5.6770e-06 - accuracy: 1.0000
Epoch 2/5
48/48 [==============================] - 8s 162ms/step - loss: 5.9981e-06 - accuracy: 1.0000
Epoch 3/5
48/48 [==============================] - 8s 163ms/step - loss: 7.9153e-06 - accuracy: 1.0000
Epoch 4/5
48/48 [==============================] - 8s 163ms/step - loss: 6.1803e-06 - accuracy: 1.0000
Epoch 5/5
48/48 [=

In [ ]:
from sklearn.metrics import confusion_matrix

pred = model.predict(testeX)
Y_pred = pred.argmax(axis=-1)

In [ ]:
cm = confusion_matrix(testeY, Y_pred)
print(cm)

In [ ]:
qualidade = cm.diagonal()/cm.sum(axis=1)
qualidade

In [ ]:
import matplotlib.pyplot as plt

grupos = ['cly', 'hook', 'lat', 'palm', 'spher', 'tip']
plt.bar(grupos, qualidade)
plt.xlabel('Movimentos')
plt.ylabel('%')
plt.title('Porcentagem de Acertos na Classificação dos Movimentos')
plt.show()

In [ ]:
desvio = np.std(qualidade)
desvio

In [ ]:
df_cm = pd.DataFrame(cm, range(6),range(6))

In [ ]:
pip install pretty_confusion_matrix

In [ ]:
import numpy as np
from pretty_confusion_matrix import pp_matrix

#pp_matrix_from_data(y_test, y_pred)
pp_matrix(df_cm)